In [ ]:
import tensorflow as tf
import time
import graph
import coarsening

#import models
%run -n models.ipynb

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Learning.
flags.DEFINE_float('num_epochs', 10, 'Number of training epochs.')
# 0.1 for cnn2, 0.3 for fgcnn2, 0.2 for lgcnn2
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_float('decay_rate', 0.95, 'Base of exponential decay. No decay with 1.')
flags.DEFINE_float('momentum', 0.9, 'Momentum. 1 indicates no momentum.')

# Regularizations.
flags.DEFINE_float('regularization', 5e-4, 'L2 regularizations of weights and biases.')
flags.DEFINE_float('dropout', 0.5, 'Dropout regularization (fc layers): probability to keep hidden neurons.'
                  'Deactivate with 1.')

flags.DEFINE_integer('batch_size', 100, 'Batch size. Must divide evenly into the dataset sizes.')
flags.DEFINE_integer('eval_frequency', 300, 'Number of steps between evaluations.')

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Feature graph

In [ ]:
def grid_graph(m, corners=False):
    z = graph.grid(m)
    A = graph.adjacency(z, k=FLAGS.number_edges)

    # Connections are only vertical or horizontal on the grid.
    # Corner vertices are connected to 2 neightbors only.
    if corners:
        import scipy.sparse
        A = A.toarray()
        A[A < A.max()/1.5] = 0
        A = scipy.sparse.csr_matrix(A)
        print('{} edges'.format(A.nnz))

    print("{} > {} edges".format(A.nnz, FLAGS.number_edges*m**2))
    return A

def coarsen(A, levels):
    graphs, parents = coarsening.metis(A, levels)
    perms = coarsening.compute_perm(parents)

    laplacians = []
    for i,A in enumerate(graphs):
        M, M = A.shape

        # No self-connections.
        if True:
            A = A.tocoo()
            A.setdiag(0)

        if i < levels:
            A = coarsening.perm_adjacency(A, perms[i])

        A = A.tocsr()
        A.eliminate_zeros()
        Mnew, Mnew = A.shape
        print('Layer {0}: M_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz))

        L = graph.laplacian(A, normalized=FLAGS.normalized_laplacian)
        laplacians.append(L)
    return laplacians, perms[0] if len(perms) > 0 else None

t_start = time.process_time()
A = grid_graph(28, corners=False)
L, perm = coarsen(A, FLAGS.coarsening_levels)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
del A

if False:
    for i,lap in enumerate(L):
        lamb, U = graph.fourier(lap)
        print('L_{}: spectrum in [{:1.2e}, {:1.2e}]'.format(i, lamb[0], lamb[-1]))

# Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data_mnist", one_hot=False)

t_start = time.process_time()
mnist.train._images = coarsening.perm_data(mnist.train._images, perm)
mnist.validation._images = coarsening.perm_data(mnist.validation._images, perm)
mnist.test._images = coarsening.perm_data(mnist.test._images, perm)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
del perm

# Neural network

In [ ]:
def evaluate(sess, op_ncorrects, op_loss, data, labels):
    """
    Runs one evaluation against the full epoch of data.
    Return the precision and the number of correct predictions.
    Batch evaluation saves memory and enables this to run on smaller GPUs.
    
    sess: the session in which the model has been trained.
    op: the Tensor that returns the number of correct predictions.
    data: size N x M
        N: number of signals (samples)
        M: number of vertices (features)
    labels: size N
        N: number of signals (samples)
    """
    ncorrects = 0  # Counts the number of correct predictions.
    loss = 0
    size = data.shape[0]
    for begin in range(0, size, FLAGS.batch_size):
        end = begin + FLAGS.batch_size
        batch_data, batch_labels = data[begin:end,:], labels[begin:end]
        feed_dict = {ph_data: batch_data, ph_labels: batch_labels, 'dropout:0': 1}
        batch_ncorrects, batch_loss = sess.run([op_ncorrects, op_loss], feed_dict)
        ncorrects += batch_ncorrects
        loss += batch_loss
    precision = ncorrects / size
    loss /= size
    return 'precision: {:.2f}% ({:d} / {:d}), loss: {:.2e}'.format(precision*100, ncorrects, size, loss)

In [ ]:
model = fc1()
model = fc2(nhiddens=100)
model = cnn2(K=5, F=10)  # K=28 is equivalent to filtering with fgcnn.
model = fcnn2(F=10)
model = fgcnn2(L[0], F=10)
model = lgcnn2_2(L[0], F=10, K=10)
model = cgcnn(L, F=[10], K=[20], p=[1], M=[])  # --> cgcnn2_2(L[0], F=10, K=20)
#model = cgcnn2_3(L[0], F=10, K=5)
#model = cgcnn2_4(L[0], F=10, K=5)
#model = cgcnn2_5(L[0], F=10, K=5)

# Architecture of TF MNIST conv model (LeNet-5-like).
# Changes: regularization, dropout, decaying training rate, momentum optimizer, stopping condition.
# Differences: training data randomization, init conv1 biases at 0.
model = cgcnn(L, F=[32,64], K=[25,25], p=[4,4], M=[512])

if False:
    K = 5  # 5 or 5^2
    t_start = time.process_time()
    mnist.test._images = graph.lanczos(L, mnist.test._images.T, K).T
    mnist.train._images = graph.lanczos(L, mnist.train._images.T, K).T
    model = lgcnn2_1(L, F=10, K=K)
    print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
    ph_data = tf.placeholder(tf.float32, (FLAGS.batch_size, mnist.train.images.shape[1], K), 'data')
else:
    ph_data = tf.placeholder(tf.float32, (FLAGS.batch_size, mnist.train.images.shape[1]), 'data')

In [ ]:
ph_labels = tf.placeholder(tf.int32, (FLAGS.batch_size), 'labels')

# Construct computational graph.
op_logits = model.inference(ph_data)
op_loss = model.loss(op_logits, ph_labels, FLAGS.regularization)
op_train, op_learning_rate = model.training(op_loss, FLAGS.learning_rate,
        mnist.train.num_examples/FLAGS.batch_size, FLAGS.decay_rate, FLAGS.momentum)
op_ncorrects = model.evaluation(op_logits, ph_labels)

# Initialize variables, i.e. weights and biases.
t_start = time.process_time()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Training.
num_steps = int(FLAGS.num_epochs * mnist.train.num_examples / FLAGS.batch_size)
for step in range(1, num_steps+1):
    batch_data, batch_labels = mnist.train.next_batch(FLAGS.batch_size)
    feed_dict = {ph_data: batch_data, ph_labels: batch_labels, 'dropout:0': FLAGS.dropout}
    learning_rate, _ = sess.run([op_learning_rate, op_train], feed_dict)
    
    # Periodical evaluation of the model.
    if step % FLAGS.eval_frequency == 0 or step == num_steps:
        print('step {} / {} (epoch {:.2f} / {}), learning_rate={:.2e}:'.format(
                step, num_steps, step*FLAGS.batch_size/mnist.train.num_examples, FLAGS.num_epochs, learning_rate))
        print('  minibatch {}'.format(evaluate(sess, op_ncorrects, op_loss, batch_data, batch_labels)))
        print('  validation {}'.format(evaluate(
                sess, op_ncorrects, op_loss,mnist.validation.images, mnist.validation.labels)))
        print('  time: {:.0f}s'.format(time.process_time() - t_start))

In [ ]:
# Evaluate.
t_start = time.process_time()
print('test {}'.format(evaluate(sess, op_ncorrects, op_loss, mnist.test.images, mnist.test.labels)))
print('time: {:.2f}s'.format(time.process_time() - t_start))
#sess.close()